In [109]:
!pip install implicit

    100% |████████████████████████████████| 1.1MB 1.3MB/s ta 0:00:01
  Stored in directory: /home/benito/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [110]:
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, SVD
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise import accuracy
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import scipy.sparse as sparse

from implicit.als import AlternatingLeastSquares

Требуется построить модель рекомендаций на основе скрытых факторов (implicit) на основе dataset’а https://grouplens.org/datasets/hetrec-2011/ (Delicious Bookmarks)

Насколько я поняла, файлы user_taggedbookmarks описывает процесс проставления тегов разным урлам разными пользователями.
На основе этой информации можно рекомендовать пользователями новые урлы.
Проставленный тег - это скрытый фактор.

Просмотрев теги становится понятно, что они описывают тематику содержания, а не оценку качества содержания.

In [149]:
user_taggedbookmarks = pd.read_csv('/home/benito/Desktop/Рекомендации на основе скрытых факторов/hetrec2011-delicious-2k/user_taggedbookmarks.dat')

In [150]:
def new_df(data):
    col=data.columns.tolist()
    col=col[0]
    new_df = data[col].str.split('\t',expand=True)
    cols=data.columns.tolist()
    cols=cols[0].split('\t')
    new_df.columns=cols
    return new_df

In [151]:
df=new_df(user_taggedbookmarks)

In [152]:
df.head()

,userID,bookmarkID,tagID,day,month,year,hour,minute,second
0,8,1,1,8,11,2010,23,29,22
1,8,2,1,8,11,2010,23,25,59
2,8,7,1,8,11,2010,18,55,1
3,8,7,6,8,11,2010,18,55,1
4,8,7,7,8,11,2010,18,55,1


Видно, что один пользователь может поставить много тегов одному урлу.
Это значит, что в модели мы будем использовать сам факт проставления тега.

In [153]:
del df['day']
del df['month']
del df['year']
del df['hour']
del df['second']
del df['minute']

In [155]:
if df['tagID'].isna().any():
    print('yes')
else:
    print('no')

no


Пустых тегов нет, значит, все пары userID 	bookmarkID говорят о том, что пользователь поставил тег закладке. Тогда нам достаточно удалить столбец tagID и создать столбец tag, равный единице, который будет говорить о том, что пользователь поставил тег закладке. Потом удалим все одинаковые строки

In [156]:
df['tag']=1

In [160]:
del df['tagID']

In [200]:
df=df.drop_duplicates()

In [208]:
df.head()

,userID,bookmarkID,tag
0,8,1,1
1,8,2,1
2,8,7,1
5,8,8,1
8,8,9,1


In [232]:
users=list(df.userID.unique())
items=list(df.bookmarkID.unique())
tags=list(df.tag)

In [241]:
rows=df.userID.astype(int)
cols=df.bookmarkID.astype(int)

In [243]:
print(len(users),len(items))

1867 69223


In [233]:
df_sparse=sparse.csr_matrix((tags,(cols,rows)))

In [234]:
df_sparse.get_shape()

(107254, 108036)

In [248]:
#df_sparse=sparse.csr_matrix((tags,(cols,rows)),shape=(len(items),len(users))) 
# не работает: row index exceeds matrix dimensions

In [236]:
model=AlternatingLeastSquares(factors=50)

In [237]:
model.fit(df_sparse)

In [239]:
userID=8

user_items=df_sparse.T.tocsr()
recomm=model.recommend(userID, user_items)

In [240]:
recomm

[(2178, 0.13074425),
 (4638, 0.09356825),
 (10742, 0.084930435),
 (14121, 0.0806653),
 (11276, 0.078226075),
 (11253, 0.07800911),
 (1631, 0.07666272),
 (11300, 0.07647579),
 (11258, 0.07618757),
 (24141, 0.07218067)]